In [1]:
# setup
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('../rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})


/var/folders/58/tt1ykq5j7t70x29f1wjcb1640000gn/T/ipykernel_32622/1251851185.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display,HTML


# CMPS 2200
# Introduction to Algorithms

## Introduction to Graphs


## Graph examples

- social networks
- Web link graph
- transportation
- gas pipelines
- disease spread

<img src="figures/transportation.png" width="50%"/>





## Terminology

A graph $G$ consists of
- a set of $n$ vertices/nodes $V$ 
- a set of $m$ edges $E \subseteq V \times V$

A **graph** is way to represent objects and their relations
  - **Node:** represents an object
  - **Edge:** represents a relation between two nodes. 
  - **Neighbor:** Two nodes are *neighbors* if they are connected by an edge.
- **Directed Graph:** Represents asymmetric (one-way) relationships
- **Undirected Graph:** Represents symmetric relationships



<img src="figures/graph.png" width="50%"/>

[Source](https://github.com/iit-cs579/main/blob/master/read/ek-02.pdf)



## Graphs queries

What are some things you might want to know about a graph? E.g., consider Facebook friendship graph, transport network, etc.



- Which are neighbor cities with New Orleans?
  - find **neighbors** of a node: $$~~~~~~~~~~~~~~~~~~~~~~~~~~N(v) =\{v_i ~ \mid (v, v_i) \in E ~~ \hbox{or} ~~(v_i, v) \in E\}$$
  - in-neighbors or out-neighbors for directed graph

- How popular is New Orleans?
  - **degree**: number of neighbors of a node:  $~~~~~~|N(v)|$
 


- Do I know which city is connected with New Orleans?
  - **path**: a sequence of nodes in which each consecutive pair are neighbors
  - **reachability**: $a$ is reachable from $b$ if there is a path between them

<img src="figures/city.png" width="75%"/>



- More??
  - a graph is **connected** if there is a *path* between each pair of nodes
  - **connected component:** A maximal subset of nodes such that each pair of nodes is connected 


<img src="figures/components.png" width=30%/>


[Source](https://github.com/iit-cs579/main/blob/master/read/ek-02.pdf)

### What data structures can we use to represent a graph?

### Adjacency matrix

<img src="figures/graph.png" width="50%"/>


$$
\begin{bmatrix}
  & A & B & C & D\\
A & 0 & 1 & 0 & 0\\
B & 1 & 0 & 1 & 1\\
C & 0 & 1 & 0 & 1\\
D & 0 & 1 & 1 & 0\\
\end{bmatrix}
$$

$$
\begin{bmatrix}
  & A & B & C & D\\
A & 0 & 1 & 0 & 0\\
B & 0 & 0 & 1 & 0\\
C & 0 & 0 & 0 & 1\\
D & 0 & 1 & 0 & 0\\
\end{bmatrix}
$$

In [1]:
import numpy as np
from tabulate import tabulate
labels = ['A', 'B', 'C', 'D']
nodes = [0,1,2,3]
edges = [(0,1), (1,2), (2, 3), (3, 2)]

def make_graph(nodes, edges, directed=False):
    graph = np.zeros((len(nodes), len(nodes)), dtype=int)
    for e in edges:
        graph[e[0], e[1]] = 1
        if not directed: # add reverse direction
            graph[e[1], e[0]] = 1
    return graph
        
graph = make_graph(nodes, edges, directed=False)
print('undirected')
print(tabulate(graph, labels))
      
digraph = make_graph(nodes, edges, directed=True)
print('\ndirected')
print(tabulate(digraph, labels))

undirected
  A    B    C    D
---  ---  ---  ---
  0    1    0    0
  1    0    1    0
  0    1    0    1
  0    0    1    0

directed
  A    B    C    D
---  ---  ---  ---
  0    1    0    0
  0    0    1    0
  0    0    0    1
  0    0    1    0


### Find Neighbors

In [2]:
def neighbors_adjacency(graph, node, labels):
    result = []
    i = labels.index(node) 
    l_i = len(graph[i])
    for j in range(l_i):
        if graph[i,j] != 0:
            result.append(labels[j])
    return result
    
print(neighbors_adjacency(graph, 'B', labels))

print(neighbors_adjacency(digraph, 'B', labels))

['A', 'C']
['C']


Runtime to access neighbors of a node?

$O(|V|)$

### What is the problem of adjacency matrix in representing a graph?

### Space Inefficient

If there are $|V|$ nodes, what is the maximum number of edges?




$$\frac{|V|(|V|-1)}{2} \in O(|V|^2)$$


....but, if a graph is dense, then there's not really any value in representing the data as a graph.

Luckily, most real-world graphs are extremely sparse.




## Edge Sets

We could simply store the graph as a set of edges $E$. [**Sparse Graph**]

In [2]:
edges = set([('A', 'B'),
             ('B', 'C'),
             ('C', 'D'),
             ('D', 'B')])
edges

{('A', 'B'), ('B', 'C'), ('C', 'D'), ('D', 'B')}

How can we access the neighbors of a node?

In [3]:
def neighbors_set(edges, node):    
    # assuming an undirected graph
    result = []
    for e in edges:
  
    return result

neighbors_set(edges, 'B')

['C', 'D', 'A']

What is the work/span of accessing neighbors using an edge set? [Considering parallelism.]

Work: $O(|E|)$

Span: $O(\lg |E|)$

What is the relationship between $|E|$ and $|V|$?

<br><br>

Can we do better?



# Map of Neighbors

We can use a hashmap (`dict`) to store the neighbors of each node.


In [2]:
graph = {
            'A': {'B'},
            'B': {'A', 'C', 'D'},
            'C': {'B', 'D'},
            'D': {'B', 'C'}
        }
graph

{'A': {'B'}, 'B': {'A', 'C', 'D'}, 'C': {'B', 'D'}, 'D': {'B', 'C'}}

In [10]:
def neighbors_map(graph, node):
    return graph[node]

neighbors_map(graph, 'B')

{'A', 'B', 'D'}

Constant time to lookup the neighbors of a node.

But, to enumerate all the neighbors, it could take $|V|$ in the worst-case.

So, same as the Adjacency Matrix, but with $O(|E|)$ space, instead of $O(|V|^2)$



## Graph Search

One of the fundamental operations over graphs

- Start at a *source* node *s*
- Visit all *reachable* nodes 
  - $t$ is reachable from $s$ if there is a path between them
- For efficiency, visit each node at most once.


<br><br>
Can be used to solve a number of problem:

- Is the graph *connected*?
- Is node $t$ reachable from node $s$
- Shortest path from $s$ to $t$


## Generic Graph Search

Consider the task of crawling every web page reachable from a starting page $s$.

How would you do this?



At any point in the search, a vertex can be in one of three sets:

- **visited**: the set of vertices already visited
- **frontier**: the unvisited neighbors of the visited vertices
- **unseen**: everything else

We can then describe a generic search algorithm as follows:

> while vertices remain to be visited:
- visit some unvisited nodes in the frontier
- update the three sets


<img src="figures/components.png" width=30%/>


```python 

def generic_search(graph, start_node, get_next_node_fn):
    visited = set([start_node])
    frontier = set([start_node])
    while len(frontier) != 0:
        print('frontier', frontier)
        ## pick a node
        node = get_next_node_fn(frontier)
        print('visiting', node)
        
        ## update visited set
        visited.add(node)
        
        ## update frontier set
        for ni in graph[node]:
            if ni not in visited:
                frontier.add(ni)
                
    return visited


```

Given a graph:
- Is the graph *connected*?
- Is node $t$ reachable from node $s$